<a href="https://colab.research.google.com/github/mmender2/DataScience/blob/main/TwoTowerLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""!pip install catboost
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install sklearn
!pip install tqdm"""
!pip install tensorflow-recommenders


In [20]:
import os, sys, time, copy
from pathlib import Path
from tqdm.auto import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn import manifold
from sklearn import decomposition
from sklearn import pipeline
from sklearn import model_selection
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import random
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras import layers

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [162]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
# select numerical and categorical features
num_cols = train.select_dtypes(include=np.number).columns.tolist()
cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()
if 'Id' in num_cols:
    num_cols.remove('Id')
print(f"Total numerical features: {len(num_cols)}")
print(f"Total categorical features: {len(cat_cols)}")
print(num_cols)
print(cat_cols)

Train shape: (1460, 81)
Test shape: (1459, 80)
Total numerical features: 37
Total categorical features: 43
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'Centr

In [99]:
def summary(df):
    print(f"Data shape: {df.shape}")
    print("\nSummary of columns missing values")
    print(df[df.isna().sum()[df.isna().sum() > 0].index].dtypes)
    print()
    summ = pd.DataFrame(df.dtypes, columns=['Data type'])
    summ['#missing'] = df.isnull().sum().values
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values

    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['first quartile'] = desc.loc[:, '25%'].values
    summ['second quartile'] = desc.loc[:, '50%'].values
    summ['third quartile'] = desc.loc[:, '75%'].values

    return summ

In [163]:
std_scaler = preprocessing.StandardScaler()
mm_scaler = preprocessing.MinMaxScaler()
robust_scaler = preprocessing.RobustScaler()
quantile_scaler = preprocessing.QuantileTransformer(n_quantiles=60, output_distribution='normal')

In [164]:
scaled_train_df = train.copy(deep=True)
scaled_test_df = test.copy(deep=True)
scaled_train_df[num_cols] = mm_scaler.fit_transform(scaled_train_df[num_cols])
num_cols.remove('SalePrice')
scaled_test_df[num_cols] = mm_scaler.fit_transform(scaled_test_df[num_cols])

In [165]:
summary(scaled_train_df)

Data shape: (1460, 81)

Summary of columns missing values
LotFrontage     float64
Alley            object
MasVnrType       object
MasVnrArea      float64
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Electrical       object
FireplaceQu      object
GarageType       object
GarageYrBlt     float64
GarageFinish     object
GarageQual       object
GarageCond       object
PoolQC           object
Fence            object
MiscFeature      object
dtype: object



,Data type,#missing,%missing,#unique,min,max,first quartile,second quartile,third quartile
Id,int64,0,0.000000,1460,1.0,1460.0,365.75,730.5,1095.25
MSSubClass,float64,0,0.000000,15,0.0,1.0,0.0,0.176471,0.294118
MSZoning,object,0,0.000000,5,NaN,NaN,NaN,NaN,NaN
LotFrontage,float64,259,17.739726,110,0.0,1.0,0.130137,0.164384,0.202055
LotArea,float64,0,0.000000,1073,0.0,1.0,0.029229,0.038227,0.04815
...,...,...,...,...,...,...,...,...,...
MoSold,float64,0,0.000000,12,0.0,1.0,0.363636,0.454545,0.636364
YrSold,float64,0,0.000000,5,0.0,1.0,0.25,0.5,0.75
SaleType,object,0,0.000000,9,NaN,NaN,NaN,NaN,NaN
SaleCondition,object,0,0.000000,6,NaN,NaN,NaN,NaN,NaN


In [264]:
X = scaled_train_df.drop('SalePrice', axis = 1)
X = X.drop('Id', axis=1)
X = scaled_train_df.dropna(thresh= len(X)*.8, axis=1)
#X = scaled_train_df.dropna()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
imputer = SimpleImputer(strategy='most_frequent')  # You can choose different imputation strategies like 'mean', 'median', 'most_frequent', etc.
Xnum = pd.DataFrame(imputer.fit_transform(X[num_cols]), columns = X[num_cols].columns)
Xcat = pd.DataFrame(imputer.fit_transform(X[cat_cols]), columns = X[cat_cols].columns)
y = scaled_train_df[['SalePrice']]

XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.3)
timeSteps = 3
# Encode categorical columns in X_train
catTrain = []
catTest = []
# Define the encoder
encoder = OneHotEncoder(handle_unknown='ignore')  # 'ignore' helps to handle categorical levels present in test but not in train
# Fit the encoder on training data
if ('SalePrice' in cat_cols):
  cat_cols.remove('SalePrice')
encoder.fit(XTrain[cat_cols])

# Transform both training and test datasets
catTrain = encoder.transform(XTrain[cat_cols])
catTest = encoder.transform(XTest[cat_cols])

#catTrain = pd.get_dummies(XTrain[cat_cols], drop_first=False)
#catTest = pd.get_dummies(XTest[cat_cols], drop_first=False)


catTrain_df = pd.DataFrame(catTrain.toarray(), columns=encoder.get_feature_names_out(input_features=cat_cols))
catTest_df = pd.DataFrame(catTest.toarray(), columns=encoder.get_feature_names_out(input_features=cat_cols))
##catTrain_df = catTrain
#catTest_df = catTest

#catTrain_df = catTrain_df.dropna()
#catTest_df = catTest_df.dropna()
# Select numerical columns
numTrain = XTrain[num_cols]
numTest = XTest[num_cols]
#numTrain_df = numTrain.dropna()
#numTest_df = numTest.dropna()
combined_df = pd.concat([numTrain, catTrain_df, yTrain], axis=1, keys=['num', 'cat', 'y'])
combined_df_clean = combined_df.dropna()
numTrain = combined_df_clean['num']
catTrain = combined_df_clean['cat']
yTrain = combined_df_clean['y']

combined_df = pd.concat([numTest, catTest_df, yTest], axis=1, keys=['num', 'cat', 'y'])
combined_df_clean = combined_df.dropna()
numTest = combined_df_clean['num']
catTest = combined_df_clean['cat']
yTest = combined_df_clean['y']

numTrainR = np.array([numTrain[i:i + timeSteps] for i in range(len(numTrain) - timeSteps + 1)])
numTestR = np.array([numTest[i:i + timeSteps] for i in range(len(numTest) - timeSteps + 1)])
catTrainR = np.array([catTrain[i:i + timeSteps] for i in range(len(catTrain) - timeSteps + 1)])
catTestR = np.array([catTest[i:i + timeSteps] for i in range(len(catTest) - timeSteps + 1)])
#yTrainB = np.where(yTrain == ' >50K', 1, 0)
#yTestB = np.where(yTest == ' >50K', 1, 0)
yTrainReshaped = yTrain[timeSteps - 1 :]
yTestReshaped = yTest[timeSteps - 1 :]


layer_sizesC=[timeSteps, catTrainR.shape[2]]
layer_sizesN=[timeSteps, numTrainR.shape[2]]

In [281]:
#summary(numTrain)
numTest.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1461,0.000000,0.329609,0.184147,0.444444,0.625,0.625954,0.183333,0.000000,0.116708,...,0.490591,0.098315,0.000000,0.0,0.0,0.208333,0.0,0.000000,0.454545,1.0
1,1462,0.000000,0.335196,0.232124,0.555556,0.625,0.603053,0.133333,0.083721,0.230175,...,0.209677,0.275983,0.048518,0.0,0.0,0.000000,0.0,0.735294,0.454545,1.0
2,1463,0.235294,0.296089,0.224197,0.444444,0.500,0.900763,0.800000,0.000000,0.197257,...,0.323925,0.148876,0.045822,0.0,0.0,0.000000,0.0,0.000000,0.181818,1.0
3,1464,0.235294,0.318436,0.154326,0.555556,0.625,0.908397,0.800000,0.015504,0.150125,...,0.315860,0.252809,0.048518,0.0,0.0,0.000000,0.0,0.000000,0.454545,1.0
4,1465,0.588235,0.122905,0.064121,0.777778,0.500,0.862595,0.700000,0.000000,0.065586,...,0.340054,0.000000,0.110512,0.0,0.0,0.250000,0.0,0.000000,0.000000,1.0


In [234]:
class HouseFeatureTower(tf.keras.layers.Layer):
    def __init__(self, layer_sizes):
        super().__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.LSTM(64, input_shape=(layer_sizes[0], layer_sizes[1]), return_sequences=True),
            tf.keras.layers.Dense(units=32, activation='relu'),
            tf.keras.layers.Dense(1, activation='relu')
])

    def call(self, inputs):
        print(f'HouseFeatureTower input shape: {inputs.shape}')  # Add this line
        return self.encoder(inputs)
        print(f'output shape in HouseFeatureTower call: {output.shape}')

In [235]:
class SalePriceTower(tf.keras.layers.Layer):
    def __init__(self, layer_sizes):
        super().__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.LSTM(64, input_shape=(layer_sizes[0], layer_sizes[1]), return_sequences=True),
            tf.keras.layers.Dense(units=32, activation='relu'),
            tf.keras.layers.Dense(1, activation='relu')
])
    def call(self, inputs):
        print(f'Q input shape: {inputs.shape}')  # Add this line
        return self.encoder(inputs)
        print(f'output shape in Q call: {output.shape}')

In [274]:


class HousePricePredictionModel(tfrs.models.Model):
    def __init__(self, layer_sizesN, layer_sizesC):
        super(HousePricePredictionModel, self).__init__()
        self.query_tower = HouseFeatureTower(layer_sizesN)
        self.candidate_tower = SalePriceTower(layer_sizesC)
        self.lstm1N = layers.LSTM(64, return_sequences=False)  # LSTM for the output of query_tower
        self.lstm1C = layers.LSTM(32, return_sequences=False)  # LSTM for the output of candidate_tower

        self.concatenate = layers.Concatenate(axis=-1)

        self.dense = layers.Dense(1, activation='relu')  # Adjust the number of units and activation function as necessary

        self.final_dense = layers.Dense(1, activation='relu')

    def compute_loss(self, y_true, y_pred):
        # Use binary cross-entropy as the loss function
        y_pred = tf.keras.activations.relu(y_pred)
        loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
        return loss


    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            y_pred = tf.reduce_mean(y_pred, axis=1)
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            y = tf.reshape(y, [-1, 1])
            loss = self.compute_loss(y, y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}



    def test_step(self, data):
        # Unpack the data
        x, y_true = data

        # Compute predictions
        y_pred = self(x, training=False)

        # Updates the metrics tracking the loss
        self.compiled_loss(y_true, y_pred, regularization_losses=self.losses)

        # Update the metrics.
        self.compiled_metrics.update_state(y_true, y_pred)

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}
    def call(self, inputs, training=False):
        q = inputs[0]
        c = inputs[1]
        query_embeddings = self.query_tower(q)
        candidate_embeddings = self.candidate_tower(c)
        print("Query embeddings shape:", query_embeddings.shape)
        print("Candidate embeddings shape:", candidate_embeddings.shape)
        # Expand dimensions to make the output 3D for LSTM layer
        #query_embeddings = tf.expand_dims(query_embeddings, 1)
        #candidate_embeddings = tf.expand_dims(candidate_embeddings, 1)

        combined = self.concatenate([query_embeddings, candidate_embeddings])
        x = self.dense(combined)
        output = self.final_dense(x)
        return output


In [275]:
# Instantiate the model
model = HousePricePredictionModel(layer_sizesN, layer_sizesC)

In [276]:
# Compile the model

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])


In [277]:
model.fit([numTrainR, catTrainR], yTrainReshaped, epochs=10, verbose=1, validation_data=([numTestR,catTestR], yTestReshaped))


Epoch 1/10
HouseFeatureTower input shape: (None, 3, 36)
Q input shape: (None, 3, 240)
Query embeddings shape: (None, 3, 1)
Candidate embeddings shape: (None, 3, 1)
HouseFeatureTower input shape: (None, 3, 36)
Q input shape: (None, 3, 240)
Query embeddings shape: (None, 3, 1)
Candidate embeddings shape: (None, 3, 1)
18/18 [==============================] - ETA: 0s - mse: 0.0609HouseFeatureTower input shape: (None, 3, 36)
Q input shape: (None, 3, 240)
Query embeddings shape: (None, 3, 1)
Candidate embeddings shape: (None, 3, 1)
18/18 [==============================] - 8s 74ms/step - mse: 0.0609 - val_loss: 0.0584 - val_mse: 0.0584
Epoch 2/10
18/18 [==============================] - 0s 13ms/step - mse: 0.0609 - val_loss: 0.0584 - val_mse: 0.0584
Epoch 3/10
18/18 [==============================] - 0s 11ms/step - mse: 0.0609 - val_loss: 0.0584 - val_mse: 0.0584
Epoch 4/10
18/18 [==============================] - 0s 12ms/step - mse: 0.0609 - val_loss: 0.0584 - val_mse: 0.0584
Epoch 5/10
18/1

In [290]:
# Encode categorical columns in X_train
numTest = []
catTest = []
# Define the encoder
encoder = OneHotEncoder(handle_unknown='ignore')  # 'ignore' helps to handle categorical levels present in test but not in train

# Fit the encoder on training data
if ('class' in cat_cols):
  cat_cols.remove('class')
encoder.fit(scaled_test_df[cat_cols])
# Transform both training and test datasets
catTest = encoder.transform(scaled_test_df[cat_cols])
catTest_df = pd.DataFrame(catTest.toarray(), columns=encoder.get_feature_names_out(input_features=cat_cols))


missing_col = set(catTrain_df.columns) - set(catTest_df.columns)
for col in missing_col:
    catTest_df[col] = 0

num_cols = scaled_test_df.select_dtypes(include=[np.number]).columns.tolist()
# Select numerical columns
numTest = scaled_test_df[num_cols]
numTest = numTest.drop(columns=['Id'])
numTestR = np.array([numTest[i:i + timeSteps] for i in range(len(numTest) - timeSteps + 1)])
catTestR = np.array([catTest_df[i:i + timeSteps] for i in range(len(catTest_df) - timeSteps + 1)])

#yTestB = np.where(scaled_test_df['class'] == ' >50K', 1, 0)


In [291]:
numTest.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.000000,0.329609,0.184147,0.444444,0.625,0.625954,0.183333,0.000000,0.116708,0.094364,...,0.490591,0.098315,0.000000,0.0,0.0,0.208333,0.0,0.000000,0.454545,1.0
1,0.000000,0.335196,0.232124,0.555556,0.625,0.603053,0.133333,0.083721,0.230175,0.000000,...,0.209677,0.275983,0.048518,0.0,0.0,0.000000,0.0,0.735294,0.454545,1.0
2,0.235294,0.296089,0.224197,0.444444,0.500,0.900763,0.800000,0.000000,0.197257,0.000000,...,0.323925,0.148876,0.045822,0.0,0.0,0.000000,0.0,0.000000,0.181818,1.0
3,0.235294,0.318436,0.154326,0.555556,0.625,0.908397,0.800000,0.015504,0.150125,0.000000,...,0.315860,0.252809,0.048518,0.0,0.0,0.000000,0.0,0.000000,0.454545,1.0
4,0.588235,0.122905,0.064121,0.777778,0.500,0.862595,0.700000,0.000000,0.065586,0.000000,...,0.340054,0.000000,0.110512,0.0,0.0,0.250000,0.0,0.000000,0.000000,1.0


In [292]:
predictions = model((numTestR, catTestR))

HouseFeatureTower input shape: (1457, 3, 36)
Q input shape: (1457, 3, 250)


ValueError: ignored

In [ ]:
# Generate thresholds
num_thresholds = 100
thresholds = np.linspace(0, 1, num_thresholds)

# Initialize variables for optimal threshold and corresponding metric
best_f1 = 0
optimal_threshold = 0

# Iterate over thresholds and compute evaluation metrics
j = 0
for threshold in thresholds:
    # Convert probabilities into class labels based on threshold
    predicted_labels = np.where(np.array(predictions) >= threshold, 1, 0)

    # Calculate evaluation metric (e.g., F1-score) using the true labels (y_test) and predicted labels
    f1 = f1_score(yTestB, predicted_labels)
    j += 1
    # Update optimal threshold and metric if a higher F1-score is achieved
    if f1 > best_f1:
        best_f1 = f1
        optimal_threshold = threshold

print("Optimal Threshold:", optimal_threshold)
print("Best F1-score:", best_f1)


In [ ]:
precision = precision_score(yTestB, np.where(predictions > optimal_threshold, 1, 0))
recall = recall_score(yTestB, np.where(predictions > optimal_threshold, 1, 0))
f1 = f1_score(yTestB, np.where(predictions > optimal_threshold, 1, 0))
auroc = roc_auc_score(yTestB, np.where(predictions > optimal_threshold, 1, 0))
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print('AUROC Score:', auroc)